In [ ]:
import mne
import librosa
import numpy as np

class DataPreprocessing:
    def __init__(self):
        pass

    @staticmethod
    def compute_cepstrum_mel(data, sfreq, n_mfcc):
        stft = librosa.stft(data)
        power_spectrum = np.abs(stft) ** 2
        mfccs = librosa.feature.mfcc(y=data, sr=sfreq, n_mfcc=n_mfcc)
        return mfccs

    def preprocess_data(self, edf_file_path, channel_pairs_joined, n_mfcc=20, tmin=None, tmax=None):
        # Load the EDF file
        raw = mne.io.read_raw_edf(edf_file_path, preload=True)

        # Change the channel type for a specific channel
        channel_names = ['2', 'SPO2', 'HR', 'MK']
        new_channel_type = ['ecg', 'hbo', 'hbo', 'hbo']
        raw.set_channel_types(dict(zip(channel_names, new_channel_type)))

        # Filtering to remove slow drifts
        filt_raw = raw.copy().filter(l_freq=1.0, h_freq=None)

        ica = mne.preprocessing.ICA(n_components=15, max_iter="auto", random_state=97)
        ica.fit(filt_raw)

        ica.exclude = []
        ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method="correlation", threshold="auto")
        ica.exclude = ecg_indices

        reconst_raw = raw.copy()
        ica.apply(reconst_raw)

        raw_bip_ref = mne.set_bipolar_reference(reconst_raw, anode=anodes, cathode=cathodes)
        raw_bip_ref_ch = raw_bip_ref.copy().pick_channels(channel_pairs_joined)

        raw_clean = mne.preprocessing.oversampled_temporal_projection(raw_bip_ref_ch)
        raw_clean.filter(0.0, 40.0)

        data = raw_clean.get_data(picks=channel_pairs_joined, tmin=tmin, tmax=tmax)

        sfreq = raw.info['sfreq']
        cepstrum_mel_features = []
        for channel_data in data:
            cepstrum_mel = self.compute_cepstrum_mel(channel_data, sfreq, n_mfcc)
            cepstrum_mel_features.append(cepstrum_mel)

        cepstral_features = np.concatenate(cepstrum_mel_features, axis=0)

        return cepstral_features

In [ ]:
data_preprocessing = DataPreprocessing()

# Define the channel pairs 
channel_pairs = [['EEG Fp1', 'EEG F7'], ['EEG F7', 'EEG T3'], ['EEG T3', 'EEG T5'], ['EEG T5', 'EEG O1'],
                 ['EEG Fp1', 'EEG F3'], ['EEG C3', 'EEG F3'], ['EEG F3', 'EEG O1'], ['EEG Fp2', 'EEG F4'],
                 ['EEG F4', 'EEG C4'], ['EEG C4', 'EEG P4'], ['EEG P4', 'EEG O2'], ['EEG Fp2', 'EEG F8'],
                 ['EEG F8', 'EEG T4'], ['EEG T4', 'EEG T6'], ['EEG T6', 'EEG O2']]
channel_pairs_joined = ['{}-{}'.format(pair[0], pair[1]) for pair in channel_pairs]

cepstral_features = data_preprocessing.preprocess_data('Siena-Scalp-EEG-Database-v1.0.0\siena-scalp-eeg-database-1.0.0\PN00\PN00-1.edf', channel_pairs_joined, tmin=1068, tmax=1138)